In [234]:
import pandas as pd
import re

### Import

In [235]:
# Importing the signature key in two parts

df1 = pd.read_csv('data/bhr1.csv', error_bad_lines=False, sep=';')
columns = df1.columns
df2 = pd.read_csv('data/bhr2.csv', names = columns, error_bad_lines=False, sep=';')
df = pd.concat([df1, df2], ignore_index=True)


C:\Users\hanna\AppData\Local\Temp/ipykernel_16784/2652866692.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df1 = pd.read_csv('data/bhr1.csv', error_bad_lines=False, sep=';')
C:\Users\hanna\AppData\Local\Temp/ipykernel_16784/2652866692.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df2 = pd.read_csv('data/bhr2.csv', names = columns, error_bad_lines=False, sep=';')


In [236]:
df

,lev,sys,numbis,text,vw
0,1,S xxx,yyy,...Bibliotheca Hertziana,NaN
1,1,S xxx,yyy,...Systematischer Standortkatalog,NaN
2,1,A,NaN,...Handbücherei,NaN
3,2,Aa,NaN,......Allgemeine Nachschlagewerke,NaN
4,3,Aa xxx,yyy,.........Lexika,NaN
...,...,...,...,...,...
11958,5,Zt 9630,NaN,"...............idem (= Neuzeit (nach 1500)), n...",NaN
11959,5,Zt 9660,NaN,"...............idem (= Neuzeit (nach 1500)), n...",NaN
11960,5,Zt 9700,NaN,"...............19. und 20. Jh., allgemein",NaN
11961,5,Zt 9730,NaN,"...............idem (= 19. und 20. Jh.), nur k...",NaN


## Cleaning

#### Text 

In [237]:
# getting rid of the ... in the texts
df["text"] = df["text"].str.replace(r'\.{3,}', '')

C:\Users\hanna\AppData\Local\Temp/ipykernel_16784/1185189639.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"] = df["text"].str.replace(r'\.{3,}', '')


In [238]:
df.sample(10)

,lev,sys,numbis,text,vw
10843,5,Zf 412,NaN,"idem (= Antike (bis 400)), Italien",NaN
8889,5,Sk 2105,NaN,interkontinental,NaN
3764,4,Dn 3370,NaN,Chiesa S. Maria in Publicolis,NaN
344,4,Ag xxx,yyy,Gelehrtenlexika und Adreßbücher,NaN
8424,5,Sb 520,530,mitteleuropäische Länder,NaN
6745,3,No xxx,yyy,Stadtbaukunst allgemein,NaN
2403,5,Bs 4410,NaN,"zeitlich durchgehend, Italien gesamt",NaN
11892,5,Zt 7840,7850,westeuropäische Länder,NaN
2186,5,Zf 2140,NaN,"Mittelalter, Italien gesamt",NaN
2883,5,Bs 8530,NaN,Ordensklerus der griechisch- und römisch-katho...,NaN


#### Backreferencing

In [239]:
# replace idem (=...)

backreference = re.compile(r"idem\s*\(\s*=\s*(.*)\s*\)")

df["backreference"] = df["text"].str.extract(backreference, expand=False)
df["text"] = df["text"].str.replace(backreference, r"\1")



In [240]:
# find the remaining idems

idem = re.compile(r'\bidem\s*[,.]?\s*')

idem_df = df[df["text"].str.contains(idem)]
print(len(idem_df))

idem_df

9


,lev,sys,numbis,text,vw,backreference
2878,5,Bs 8516,NaN,idem Neuzeit,NaN,NaN
6017,5,Ne 1532,NaN,"idem,19. Jh.",NaN,NaN
6018,5,Ne 1534,NaN,"idem, 19. und 20. Jh.",NaN,idem
6019,5,Ne 1536,NaN,"idem, 20. Jh.",NaN,idem
10056,5,Sv 3560,NaN,"idem,.nordeuropäische Länder",NaN,NaN
10057,5,Sv 3570,NaN,"idem, osteuropäische Länder",NaN,idem
10058,5,Sv 3580,NaN,"idem, Balkan-Länder",NaN,idem
10059,5,Sv 3600,NaN,"idem, außereuropäische Länder",NaN,idem
10552,4,Ua 4220,NaN,idem. Neue Folge (1937 ff.),NaN,NaN


In [241]:
# replace the idem without backreference

for i, row in idem_df.iterrows():
    # loop over rows in the df DataFrame above the current row
    for j in range(i-1, -1, -1):

        if not idem.search(df.loc[j, "text"]):

            reference = df.iloc[j].text.split(',')[0]
            
            #replace idem with the actual backreference
            substring = df.iloc[i].text

            modified_substring = idem.sub(f'{reference} ', substring)

            df.loc[i, "text"] = modified_substring
            break



#### Level 1, 2, 3 Texts

In [244]:
# Level 3 text

df['text_3'] = ''

indices = df[df.lev == 3].index

for i in range(len(indices) - 1): 

    start = indices[i]
    end = indices[i+1] 

    df.loc[start + 1 :end - 1, 'text_3'] = df.iloc[start].text

#handle the last case seperately 
if len(indices) > 0: 
    start = indices[-1]
    end = len(df)
    df.loc[start + 1 :end - 1, 'text_3'] = df.iloc[start].text


In [245]:
# Level 2 text 

df['text_2'] = ''

indices = df[df.lev == 2].index

for i in range(len(indices) - 1): 

    start = indices[i]
    end = indices[i+1] 

    df.loc[start + 1 :end - 1, 'text_2'] = df.iloc[start].text

#handle the last case seperately 
if len(indices) > 0: 
    start = indices[-1]
    end = len(df)
    df.loc[start + 1 :end - 1, 'text_2'] = df.iloc[start].text

In [247]:
# Level 1 text 

df['text_1'] = ''

indices = df[df.lev == 1].index

for i in range(len(indices) - 1): 

    start = indices[i]
    end = indices[i+1] 

    df.loc[start + 1 :end - 1, 'text_1'] = df.iloc[start].text


#handle the last case seperately 
if len(indices) > 0: 
    start = indices[-1]
    end = len(df)
    df.loc[start + 1 :end - 1, 'text_1'] = df.iloc[start].text

#### Ranges

In [ ]:
# Appending all the ranges as rows, to make lookup operations easier later

df.numbis = df.numbis.fillna('')
range_rows = df[df.numbis.str.isdigit()]

for i, row in range_rows.iterrows():

    start = int(re.findall(r'\d+', row.sys)[0])
    end = int(row.numbis)

    for j in range(start, end): 
        new_row = row.copy()
        new_row['sys'] = new_row['sys'].replace(str(start), str(j+1))

        df = df.append(new_row, ignore_index=True)


In [251]:
df

,lev,sys,numbis,text,vw,backreference,text_3,text_2,text_1
0,1,S xxx,yyy,Bibliotheca Hertziana,NaN,NaN,,,
1,1,S xxx,yyy,Systematischer Standortkatalog,NaN,NaN,,,
2,1,A,,Handbücherei,NaN,NaN,,,
3,2,Aa,,Allgemeine Nachschlagewerke,NaN,NaN,,,Handbücherei
4,3,Aa xxx,yyy,Lexika,NaN,NaN,,Allgemeine Nachschlagewerke,Handbücherei
...,...,...,...,...,...,...,...,...,...
55496,5,Zt 8846,8850,westeuropäische Länder,NaN,NaN,Christentum,Theologie und Frömmigkeit (Weltreligionen),Nachbarwissenschaften
55497,5,Zt 8847,8850,westeuropäische Länder,NaN,NaN,Christentum,Theologie und Frömmigkeit (Weltreligionen),Nachbarwissenschaften
55498,5,Zt 8848,8850,westeuropäische Länder,NaN,NaN,Christentum,Theologie und Frömmigkeit (Weltreligionen),Nachbarwissenschaften
55499,5,Zt 8849,8850,westeuropäische Länder,NaN,NaN,Christentum,Theologie und Frömmigkeit (Weltreligionen),Nachbarwissenschaften


## Data Merging

In [272]:
# Importing the signatures and inventory date

path = 'data/csv/hertz.csv'

df_hertz = pd.read_csv(path)

df_hertz.signature.fillna('', inplace=True)


In [292]:
# Extracting signature with the format Xx 0000


df_hertz[['signature_l', 'signature_n']] = df_hertz.signature.str.extract(r'^([a-zA-Z]{2}) (\d+)', expand=False)


df_hertz['signature_clean'] = df_hertz[['signature_l', 'signature_n']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)


# Merging on the signatures

merged = df_hertz.merge(df, left_on='signature_clean', right_on='sys')

# Export

export_columns = ['id', 'signature', 'inventory_nr', 'text', 'text_3', 'text_2', 'text_1']

merged[export_columns].to_csv('data/csv/hertz_signatures.csv')